In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
import keras
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2

In [ ]:
train_dir = '../input/landmark-recognition-2021/train'
test_dir = '../input/landmark-recognition-2021/test'

In [ ]:
train=pd.read_csv('../input/landmark-recognition-2021/train.csv')
train

In [ ]:
sample=pd.read_csv('../input/landmark-recognition-2021/sample_submission.csv')
sample

In [ ]:
Name=[]
for landmark in train['landmark_id'].unique():
    Name+=[landmark]
print(len(Name))
n=len(Name)

In [ ]:
N=list(range(n))
print(n)
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
trainX0=[]
trainY0=[]
#len(train)
for i in tqdm(range(10000)):
    idt=train.loc[i,'id']+'.jpg'
    landt=train.loc[i,'landmark_id']
    path=os.path.join(train_dir,idt[0],idt[1],idt[2])
    image=cv2.imread(os.path.join(path,idt))
    image2=cv2.resize(image,dsize=(16,16),interpolation=cv2.INTER_CUBIC)
    trainX0+=[image2]
    trainY0+=[landt]    

In [ ]:
trainX=np.array(trainX0)
trainY1=pd.Series(trainY0).map(normal_mapping)
trainY2=np.array(trainY1)

In [ ]:
trainY3=to_categorical(trainY2)
X_train=np.array(trainX0).reshape(-1,16,16,3)
y_train=np.array(trainY3)

In [ ]:
trainx,testx,trainy,testy=train_test_split(X_train,y_train,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)
print(trainy.shape[1])

In [ ]:
model = Sequential()
model.add(Conv2D(32,(2,2),input_shape = (16,16,3),activation = 'relu'))
model.add(Conv2D(32,(2,2),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(trainy.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
his = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=64, verbose=2)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
testX0=[]
for i in tqdm(range(len(sample))):
    idt=sample.loc[i,'id']+'.jpg'
    path=os.path.join(test_dir,idt[0],idt[1],idt[2])
    image=cv2.imread(os.path.join(path,idt))
    image2=cv2.resize(image,dsize=(16,16),interpolation=cv2.INTER_CUBIC)
    testX0+=[image2]

In [ ]:
X_test=np.array(testX0).reshape(-1,16,16,3)

In [ ]:
pred2=model.predict(X_test)
print(pred2[0:10])

PRED=[]
for item in pred2:
    value=np.argmax(item)
    score=item[value]
    value2=reverse_mapping[value]
    PRED+=[str(value2)+' '+str(score)]
print(PRED[0:10])

In [ ]:
submt=sample
submt['landmarks']=PRED
submt.to_csv('submission.csv',index=False)
submt

#### 